# **라이브러리**

In [ ]:
import pandas as pd
import numpy as np
#시각화
import seaborn as sns 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import missingno as msno


# **데이터 구조 파악 및 가공**

In [ ]:
#데이터 불러오기
test = pd.read_csv('../input/bike-sharing-demand/test.csv')
train = pd.read_csv('../input/bike-sharing-demand/train.csv')

**데이터 구성**

|datetime|season|workingday|weather|temp|atemp|humidity|windspeed|casual|registered|count|
|---|---|---|---|---|---|---|---|---|---|---|
|날짜|계절|평일|날씨|온도|체감온도|습도|풍속|비등록자|등록자|대여수|

weather
- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 

season

- 1: spring
- 2: summer
- 3: fall
- 4: winter 

In [ ]:
#데이터 구성 확인
train.head()

In [ ]:
train.info()

In [ ]:
train.datetime

datetime의 경우 변수형태가 object로 가공이 필요한 것 확인. 

이를 년, 월, 일, 시간으로 분리하여준다. 판다스 라이브러리를 이용하여 datetime형태로 바꾸어 준다.

In [ ]:
train.datetime = pd.to_datetime(train.datetime) # object타입에서 datetime타입으로 바꿔주기

train['year'] = train['datetime'].dt.year
train['month']= train['datetime'].dt.month
train['date']= train['datetime'].dt.day
train['hour']= train['datetime'].dt.hour 
#4개의 컬럼을 새로 만들고 알맞은 값을 대입해줌

In [ ]:
train.head() #상위 n개 출력 디폴트값은 5

In [ ]:
train = train.drop(['datetime', 'casual', 'registered'], axis = 1)#axis가 1이면 열


datetime을 가공해준 후 datetime열은 필요없기 때문에 삭제하도록 한다.

또한 registered와 casual은 합했을 때 count가 되는 변수로 너무 높은 상관관계때문에 예측을 저해할 수 있기 때문에 삭제해준다.

In [ ]:
train

# **결측치 확인**

In [ ]:

print(train.isnull().sum())#결측치 확인
print('\n')
msno.matrix(train,figsize=(12,5)) #msno를 이용하여 결측치 시각화 가능


# **결측값 정제**

이 데이터에는 결측치가 존재하지 않는다.

만약 결측치가 존재한다면?
https://workingwithpython.com/howtohandlemissingvaluewithpython/

결측치 제거

dropna함수를 이용하여 제거

결측치 대체

fillna함수를 통해 대체 가능

# **이상치 확인 및 제거**

***boxplot 이상치 시각화***

In [ ]:
fig,ax = plt.subplots(ncols=2 , nrows = 2)

fig.set_size_inches(15,15)

sns.boxplot(data = train, x = "season", y = "count", ax = ax[0][0] )
sns.boxplot(data = train, x = "weather", y = "count",  ax = ax[0][1])
sns.boxplot(data = train, x = "holiday", y = "count",  ax = ax[1][0])
sns.boxplot(data = train, x = "workingday", y = "count", ax = ax[1][1])


***1. IQR(Iner Qunatier Range)***

IQR 방식은 사분위(Quantile) 개념으로부터 출발 

전체 데이터들을 오름차순으로 정렬하고, 정확히 4등분(25%, 50%, 75%, 100%)으로 나눈다.

여기서 75% 지점의 값과 25% 지점의 값의 차이를 IQR이라고 한다.

이 IQR에 1.5를 곱해서 75% 지점의 값에 더하면 최댓값, 25% 지점의 값에서 빼면 최솟값으로 결정

이 때, 결정된 최댓값보다 크거나 최솟값보다 작은 값을 이상치 라고 간주

 

In [ ]:
"""
def get_outlier(df=None, column=None, weight=1.5):
  # target 값과 상관관계가 높은 열을 우선적으로 진행
  quantile_25 = np.percentile(df[column].values, 25)
  quantile_75 = np.percentile(df[column].values, 75)

  IQR = quantile_75 - quantile_25
  IQR_weight = IQR*weight
  
  lowest = quantile_25 - IQR_weight
  highest = quantile_75 + IQR_weight
  
  outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
  return outlier_idx

# 함수 사용해서 이상치 값 삭제
outlier_idx = get_outlier(df=train, column='count', weight=1.5)

train.drop(outlier_idx, axis=0, inplace=True)
"""

In [ ]:
#train.shape

***2. 3-sigma***

정규분포로 가정하고 실제값이 평균으로 부터 ±표준편차 * 시그마계수를 벗어나면 아웃라이어로 간주.

-> |실제값 -평균| > 표준편차 * 시그마계수(3) 이면 아웃라이어

넘파이 이용


In [ ]:
train = train[np.abs(train['count'] - train['count'].mean()) <= train['count'].std()]

In [ ]:
train.shape

# **시각화**

In [ ]:
plt.figure(figsize=(10,6))
sns.distplot(train['count'])
plt.show()

In [ ]:

fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows = 4)
fig.set_size_inches(10,15)

sns.barplot(data = train, x = "season", y = "count", ax = ax1)
sns.barplot(data = train, x = "weather", y = "count",  ax = ax2)
sns.barplot(data = train, x = "holiday", y = "count",  ax = ax3)
sns.barplot(data = train, x = "workingday", y = "count",   ax = ax4)


In [ ]:
print(train[train.weather == 4])

In [ ]:
train = train.drop([5631], axis = 0)

In [ ]:

fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows = 4)
fig.set_size_inches(10,15)

sns.barplot(data = train, x = "season", y = "count", ax = ax1)
sns.barplot(data = train, x = "weather", y = "count",  ax = ax2)
sns.barplot(data = train, x = "holiday", y = "count",  ax = ax3)
sns.barplot(data = train, x = "workingday", y = "count",   ax = ax4)


In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows = 4)
fig.set_size_inches(10,15)

sns.barplot(data = train, x = "year", y = "count", ax = ax1)
sns.barplot(data = train, x = "month", y = "count",  ax = ax2)
sns.barplot(data = train, x = "date", y = "count",  ax = ax3)
sns.barplot(data = train, x = "hour", y = "count",   ax = ax4)

2012년에 수요가 증가했으며 위 season 그래프처럼 여름, 가을에 수요가 증가함을 볼 수 있다.

In [ ]:
plt.figure(figsize=(15,15))

sns.heatmap(train.corr(), annot = True)

상관계수는 보통 0.3 ~ 0.7인 경우 뚜렷한 상관관계라고 볼 수 있다.

이 히트맵에서는 hour, temp, casual, registered, humidity가 상대적으로 높은 연관성을 보이고 있음.

이것들이 count와 어떤 관계를 가지고 있는 지 살펴보기로 하자.

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows = 4)
fig.set_size_inches(18,25)

sns.regplot(data = train, x = "hour", y = "count", ax = ax1)
sns.regplot(data = train, x = "temp", y = "count",  ax = ax2)
sns.regplot(data = train, x = "windspeed", y = "count",  ax = ax3)
sns.regplot(data = train, x = "humidity", y = "count",   ax = ax4)

상관관계가 높은 조건들을 그래프로 scatter, line 모두 표현되는 regplot을 이용하여 표현.

기상에 따른 자전거 대여량 확인

hour에 따른 count에서 7,8시와 17,18시에 높은 count 발견하여 출,퇴근시간에 따라 자전거 대여량 변화가 있는 것인지 고려하게 됨.


In [ ]:
sns.relplot(data = train, x = 'hour', y = 'count', col = 'workingday', kind = 'line')

평일과 휴일의 시간에 따른 자전거대여량을 relplot 이용하여 표현

평일의 출퇴근시간과 자전거대여량 관계가 있음.